# Revised case normalization for Hirslanden Birshof 2018

Notes:

- There is no data for the hospital 'Hirslanden Birshof' in 2018



In [1]:
import sys
sys.path.insert(0, '/home/jovyan/work')

from src.revised_case_normalization.notebook_functions.global_configs import *
from src.revised_case_normalization.notebook_functions.normalize import normalize
from src.revised_case_normalization.notebook_functions.revise import revise
from src.revised_case_normalization.notebook_functions.group import group
from src.revised_case_normalization.notebook_functions.update_db import update_db

In [2]:
file_info = FileInfo(os.path.join(ROOT_DIR, 'raw_data/HI_Aarau_Birshof_ST. Anna.xlsx'),
                     'Hirslanden Klinik Birshof', '2018', 'Birshof_2018')

print(file_info)

FileInfo(path='/home/jovyan/work/src/revised_case_normalization/raw_data/HI_Aarau_Birshof_ST. Anna.xlsx', hospital_name_db='Hirslanden Klinik Birshof', year='2018', sheet='Birshof_2018')


In [3]:
revised_cases_df = normalize(file_info)
revised_cases_df

2022-11-21 08:59:37.264 | INFO     | src.revised_case_normalization.notebook_functions.normalize:normalize:38 - Read 4 cases for Hirslanden Klinik Birshof 2018
2022-11-21 08:59:37.270 | INFO     | src.revised_case_normalization.notebook_functions.normalize:normalize:73 - TYPES:
datum                    string
tranche                  string
case_id                  object
patient_id               object
kkik                     string
gender                   string
age_years                 int64
bfs_code                 string
duration_of_stay          int64
pflegetage neu           string
pccl                      int64
pccl neu                 string
old_pd                   string
primary_diagnosis        string
added_icds               string
removed_icds             string
added_chops              string
removed_chops            string
drg                      string
drg neu                  string
cw alt                   string
cw neu                   string
cw-änderung vorsc

,case_id,patient_id,gender,age_years,duration_of_stay,case_id_norm,old_pd,primary_diagnosis,bfs_code,pccl,drg,added_icds,removed_icds,added_chops,removed_chops
0,0041676790,6B6D03EC2A3F5E3C,M,76,2,41676790,M702,M702,M200,2,I27D,"[R470, I693]",[],[],[]
1,0041733296,D506D11F343B8049,M,73,3,41733296,M702,M702,M200,2,I27D,[E1191],[E1190],[],[]
2,0041764039,F7D48CC6FF8EE1B9,M,84,2,41764039,M754,M754,M200,3,I27C,"[B022, G530]",[],[],[]
3,0041860429,5816B1A17E85581F,M,73,17,41860429,M171,M171,M200,3,I43B,[D62],[],[990410],[]


# Match revised DtoD data with the BfS data from the database

In [4]:
cols_to_join = list(VALIDATION_COLS)
cols_to_join.remove(CASE_ID_COL)
cols_to_join.append(NORM_CASE_ID_COL)
# Ignore the patient ID in this dataset, because it is encrypted
cols_to_join.remove(PATIENT_ID_COL)

revised_cases, unmatched = revise(file_info, revised_cases_df, validation_cols=cols_to_join)
revised_cases

2022-11-21 08:59:38.181 | INFO     | src.service.bfs_cases_db_service:get_sociodemographics_for_hospital_year:95 - Read 3899 rows from the DB, for the hospital 'Hirslanden Klinik Birshof' in 2018


,aimedic_id,case_id,primary_diagnosis,secondary_diagnoses,primary_procedure,secondary_procedures,gender,age_years,age_days,gestation_age,duration_of_stay,ventilation_hours,grouper_admission_type,admission_date,admission_weight,grouper_discharge_type,discharge_date
0,164532,0041764039,M754,"[M6631, M758, I4288, B022, G530]",818333:L:20180530,"[809120:L:20180530, 808120:L:20180530, 807110:L:20180530, 831321:L:20180530]",M,84,0,0,2,0,01,2018-05-30,0,00,2018-06-01
1,165090,0041676790,M702,"[I1190, I2519, I480, Z950, N181, ...]",835X12:R:20180216,[],M,76,0,0,2,0,01,2018-02-16,0,00,2018-02-18
2,165589,0041733296,M702,"[X599, B956, T798, E1191]",835X12:R:20180329,[86321E:R:20180329],M,73,0,0,3,0,01,2018-03-29,0,00,2018-04-01
3,165725,0041860429,M171,"[I652, G2090, T813, Y849, T810, ...]",815421:L:20181004,"[009910::20181004, 948X40::20181011, 86321E:L:20181018, 009910::20181018, 990410::]",M,73,0,0,17,0,01,2018-10-03,0,00,2018-10-26


In [5]:
if unmatched.shape[0] > 0:
    display(unmatched)

In [6]:
revisions_update, diagnoses_update, procedures_update = group(revised_cases)
revisions_update[REVISION_DATE_COL] = '2022-12-31'

2022-11-21 08:59:38.596 | INFO     | src.revised_case_normalization.notebook_functions.group:group:86 - Grouping 4 cases ...
2022-11-21 08:59:39.580 | SUCCESS  | src.revised_case_normalization.notebook_functions.group:group:91 - Grouped 4 cases into: 4 revisions, 31 diagnoses rows, 14 procedure rows


In [7]:
revisions_update

,aimedic_id,drg,drg_cost_weight,effective_cost_weight,pccl,revision_date
0,164532,I27A,3.02,1.59,4,2022-12-31
1,165090,I27C,1.09,1.09,3,2022-12-31
2,165589,I27C,1.09,1.09,3,2022-12-31
3,165725,I43A,2.68,2.68,4,2022-12-31


In [8]:
update_db(revisions_update, diagnoses_update, procedures_update)

2022-11-21 08:59:39.598 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_revisions:259 - Trying to insert 4 cases into the 'Revisions' table ...
2022-11-21 08:59:41.265 | SUCCESS  | src.service.bfs_cases_db_service:insert_revised_cases_into_revisions:296 - Inserted 4 cases into the 'Revisions' table
2022-11-21 08:59:41.267 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_diagnoses:309 - Trying to insert 31 rows into the 'Diagnoses' table ...
2022-11-21 08:59:44.476 | SUCCESS  | src.service.bfs_cases_db_service:insert_revised_cases_into_diagnoses:347 - Inserted 31 rows into the 'Diagnoses' table
2022-11-21 08:59:44.478 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_procedures:358 - Trying to insert 14 rows into the 'Procedures' table ...
2022-11-21 08:59:46.382 | SUCCESS  | src.service.bfs_cases_db_service:insert_revised_cases_into_procedures:404 - Inserted 14 rows into the 'Procedures' table
